In [66]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
#%matplotlib inline
plt.rcParams["animation.ffmpeg_path"] = "/usr/bin/avconv"

In [67]:
m = 15
n = 20

In [68]:
# プロジェクト名
project = "synth_05_04"

In [69]:
# TM, M_N, Mは存在
# 行列と真のカテゴリを読み込む
TM = np.loadtxt(os.path.join(project, "TM.mat"), delimiter=",")

# カラースケールの設定
margin = 1
vmin = np.min(TM)-1
vmax = np.max(TM)+1

plt.imshow(TM, interpolation="none", vmin=vmin, vmax=vmax)
plt.tight_layout()
plt.savefig(os.path.join(project, "TM.png"))
plt.show()

M_N = np.loadtxt(os.path.join(project, "M_N.mat"), delimiter=",")
plt.imshow(M_N, interpolation="none", vmin=vmin, vmax=vmax)
plt.tight_layout()
plt.savefig(os.path.join(project, "M_N.png"))
plt.show()

M = np.loadtxt(os.path.join(project, "Mnp.mat"), delimiter=",")
plt.imshow(M, interpolation="none", vmin=vmin, vmax=vmax)
plt.tight_layout()
plt.savefig(os.path.join(project, "M.png"))
plt.show()

row_cat = []
with open(os.path.join(project, "row_cat"), "r") as fin:
    for line in fin:
        row_cat.append(int(line.rstrip()))

col_cat = []
with open(os.path.join(project, "col_cat"), "r") as fin:
    for line in fin:
        col_cat.append(int(line.rstrip()))

In [70]:
# Mの平均
M_mean = np.nanmean(M)

In [91]:
# パラメータリスト
# グラフなし
#gammas = [[0.001, 0, 0], [0.01, 0, 0], [0.1, 0, 0], [1, 0, 0]]
# グラフあり
#gammas = [[0.1, 0.001, 0.001], [0.1, 0.01, 0.01], [0.1, 0.1, 0.1], [0.1, 1, 1]]
gammas = [[1, 0, 0]]
rho = 0.15

In [49]:
# パラメータごとにC++で実行
import subprocess
import sys
for gamma in gammas:
    params = gamma + [rho]
    params_str = "_".join([str(param) for param in params])
    with open("prob_param", "w") as fout:
        print("\n".join([str(param) for param in params]), file=fout)
    res = subprocess.run(["mkdir", os.path.join(project, params_str)], stderr=subprocess.PIPE)
    res = subprocess.run(["./a.out", str(m), str(n), project, params_str], stderr=subprocess.PIPE)
    print(res.stderr)

b'$i: 0\n$i: 1\n$i: 2\n$i: 3\n$i: 4\n$i: 5\n$i: 6\n$i: 7\n$i: 8\n$i: 9\n$i: 10\n$i: 11\n$i: 12\n$i: 13\n$i: 14\n$i: 15\n$i: 16\n$i: 17\n$i: 18\n$i: 19\n$i: 20\n$i: 21\n$i: 22\n$i: 23\n$i: 24\n$i: 25\n$i: 26\n$i: 27\n$i: 28\n$i: 29\n$i: 30\n$i: 31\n$i: 32\n$i: 33\n$i: 34\n$i: 35\n$i: 36\n$i: 37\n$i: 38\n$i: 39\n$i: 40\n$i: 41\n$i: 42\n$i: 43\n$i: 44\n$i: 45\n$i: 46\n$i: 47\n$i: 48\n$i: 49\n$i: 50\n$i: 51\n$i: 52\n$i: 53\n$i: 54\n$i: 55\n$i: 56\n$i: 57\n$i: 58\n$i: 59\n$i: 60\n$i: 61\n$i: 62\n$i: 63\n$i: 64\n$i: 65\n$i: 66\n$i: 67\n$i: 68\n$i: 69\n$i: 70\n$i: 71\n$i: 72\n$i: 73\n$i: 74\n'
b'$i: 0\n$i: 1\n$i: 2\n$i: 3\n$i: 4\n$i: 5\n$i: 6\n$i: 7\n$i: 8\n$i: 9\n$i: 10\n$i: 11\n$i: 12\n$i: 13\n$i: 14\n$i: 15\n$i: 16\n$i: 17\n$i: 18\n$i: 19\n$i: 20\n$i: 21\n$i: 22\n$i: 23\n$i: 24\n$i: 25\n$i: 26\n$i: 27\n$i: 28\n$i: 29\n$i: 30\n'
b'$i: 0\n$i: 1\n$i: 2\n$i: 3\n$i: 4\n$i: 5\n$i: 6\n$i: 7\n$i: 8\n$i: 9\n$i: 10\n$i: 11\n$i: 12\n$i: 13\n$i: 14\n$i: 15\n$i: 16\n$i: 17\n$i: 18\n$i: 19\n$i: 20\n$i: 

In [93]:
# パラメータごとにログを解析する
for gamma in gammas:
    params = gamma + [rho]
    params_str = "_".join([str(param) for param in params])
    it = 0
    logdir_path = os.path.join(project, params_str)
    
    fig = plt.figure()
    im_list = []
    X = np.zeros((m, n), dtype=float)
    while True:
        file_path = os.path.join(logdir_path, "X"+str(it))

        if subprocess.run(["find", file_path]).returncode != 0:
            #plt.imshow(X, interpolation="none", vmin=vmin, vmax=vmax)
            #plt.tight_layout()
            #plt.savefig(os.path.join(logdir_path, "X.png"))
            #plt.show()
            # すべて閉じる
            #plt.cla()
            #plt.clf()
            #plt.close("all")
            break

        meaned_X = np.loadtxt(file_path, delimiter=",")
        X = meaned_X + M_mean
        im = plt.imshow(X, interpolation="none", vmin=vmin, vmax=vmax)
        plt.tight_layout()
        im_list.append([im])
        it += 1
    ani = animation.ArtistAnimation(fig, im_list, interval=500, repeat=False)
    ani.save(os.path.join(logdir_path, "X.gif"))
    plt.show()
    # すべて閉じる
    #plt.cla()
    #plt.clf()
    #plt.close("all")

/home/mistpc/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/matplotlib/animation.py:782: UserWarning: MovieWriter ffmpeg unavailable
  warnings.warn("MovieWriter %s unavailable" % writer)


In [92]:
# すべて閉じる
plt.cla()
plt.clf()
plt.close("all")